In [1]:
%%capture

%pip install rdkit
%pip install mordred
%pip install padelpy


In [2]:
import pandas as pd
import numpy as np


import padelpy
import mordred

from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, rdFingerprintGenerator, rdMolDescriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

from mordred import Calculator, descriptors

from padelpy import from_smiles

from sklearn.feature_selection import VarianceThreshold

# 1. Загружаем csv файл с мини-таска 1


In [3]:
df = pd.read_csv('cox2_clean_data.csv')
df.head()

,canonical_smiles,IC50_nM
0,Cc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccccc1,60.00
1,Cc1c(C=O)cc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,3230.00
2,Cc1c(COc2cccc(Cl)c2)cc(-c2ccc(S(C)(=O)=O)cc2)n...,80.00
3,Fc1ccc(-c2[nH]c(-c3ccc(F)cc3)c3c2C2CCC3CC2)cc1,0.12
4,CCc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,100000.00


# 2. Рассчитываем дескрипторы



## RDKit


In [4]:
# инициализируем калькулятор 

desc_names = [name for name, _ in Descriptors._descList]
calculator = MoleculeDescriptors.MolecularDescriptorCalculator(desc_names)

# функция для 1 мол

def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [None] * len(desc_names)  
    try:
        return calculator.CalcDescriptors(mol)
    except:
        return [None] * len(desc_names)

# применяем к нашему дфрейму
desc_values = df['canonical_smiles'].apply(compute_descriptors)
desc_df = pd.DataFrame(desc_values.tolist(), columns=desc_names)

# склеиваем 
df_rdkit = pd.concat([df[['canonical_smiles', 'IC50_nM']], desc_df], axis=1)

In [5]:
df_rdkit.head()

,canonical_smiles,IC50_nM,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,Cc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccccc1,60.00,11.568006,11.568006,0.338694,-3.165650,0.736620,11.545455,311.406,294.270,...,0,0,1,0,0,0,0,0,0,0
1,Cc1c(C=O)cc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,3230.00,13.231678,13.231678,0.225436,-3.281084,0.667384,11.480000,357.406,341.278,...,0,0,1,0,0,0,0,0,0,0
2,Cc1c(COc2cccc(Cl)c2)cc(-c2ccc(S(C)(=O)=O)cc2)n...,80.00,13.531957,13.531957,0.252198,-3.293829,0.336163,11.500000,469.965,448.797,...,0,0,1,0,0,0,0,0,0,0
3,Fc1ccc(-c2[nH]c(-c3ccc(F)cc3)c3c2C2CCC3CC2)cc1,0.12,13.353649,13.353649,0.218100,-0.218100,0.558954,21.360000,335.397,316.245,...,0,0,0,0,0,0,0,0,0,0
4,CCc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,100000.00,13.224282,13.224282,0.274991,-3.216745,0.710290,11.625000,343.423,325.279,...,0,0,1,0,0,0,0,0,0,0


In [6]:
df_rdkit.shape

(5098, 219)

In [7]:
df_rdkit.to_csv('rdkit_desc_dirty.csv', index=False)

## Mordred

In [8]:
def compute_descriptors_mordred(df, smiles_column='canonical_smiles'):
    # инициализируем калькулятор
    calc = Calculator(descriptors, ignore_3D=True)

    # смайлз в мол
    mols = [Chem.MolFromSmiles(smi) for smi in df[smiles_column]]

    # получаем дескрипторы
    desc_df = calc.pandas(mols)

    # склейка
    df_mordred = pd.concat([df, desc_df], axis=1)

    return df_mordred

In [9]:
df_mordred = compute_descriptors_mordred(df)

 48%|█████████████████████████████████████▎                                        | 2439/5098 [01:09<10:32,  4.20it/s]

C:\Users\Бахтияр\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\Бахтияр\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\Бахтияр\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\Бахтияр\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\Бахтияр\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  re

 48%|█████████████████████████████████████▍                                        | 2443/5098 [01:13<14:05,  3.14it/s]

C:\Users\Бахтияр\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 48%|█████████████████████████████████████▍                                        | 2445/5098 [01:13<13:34,  3.26it/s]

C:\Users\Бахтияр\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\Бахтияр\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████████████████████████████████████████████████████████████████████████| 5098/5098 [02:32<00:00, 33.39it/s]


In [10]:
df_mordred.head()

,canonical_smiles,IC50_nM,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,Cc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccccc1,60.00,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,28.094879,2.460737,4.844118,28.094879,...,10.068112,70.338493,311.098000,7.976872,1061,34,118.0,139.0,7.479167,4.694444
1,Cc1c(C=O)cc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,3230.00,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,31.396165,2.49776,4.886026,31.396165,...,10.225027,74.355252,357.083493,8.709353,1482,41,134.0,159.0,9.451389,5.388889
2,Cc1c(COc2cccc(Cl)c2)cc(-c2ccc(S(C)(=O)=O)cc2)n...,80.00,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,40.58866,2.501447,4.889803,40.58866,...,10.420524,82.494485,469.091470,8.850782,3028,51,172.0,202.0,10.923611,6.805556
3,Fc1ccc(-c2[nH]c(-c3ccc(F)cc3)c3c2C2CCC3CC2)cc1,0.12,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,33.98774,2.590707,5.063035,33.98774,...,10.444910,75.054918,335.148556,7.617013,1380,43,144.0,177.0,6.361111,5.277778
4,CCc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,100000.00,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,30.368863,2.472128,4.863309,30.368863,...,10.153974,72.770133,343.104228,8.169148,1348,38,128.0,151.0,8.590278,5.194444


In [11]:
df_mordred.shape

(5098, 1615)

In [12]:
df_mordred.to_csv('mordred_desc_dirty.csv', index=False)

## PaDEL

## ужасно долго грузил. попробую позже распараллелить сразу

In [ ]:
# def compute_descriptors_padelpy(df, smiles_column='canonical_smiles'):
#     # аналогично и для padel
#     smiles_list = df[smiles_column].tolist()

#     descriptors_list = [from_smiles(smi, descriptors=True, fingerprints=False) for smi in smiles_list]

#     desc_df = pd.DataFrame(descriptors_list)

#     # склеиваем
#     df_padel = pd.concat([df.reset_index(drop=True), desc_df.reset_index(drop=True)], axis=1)

#     return df_padel

#TODO

In [ ]:
# df_padel = compute_descriptors_padelpy(df)

# 3. Чистим признаки

In [13]:
def cleaning_desc(df, smiles_col='canonical_smiles', activity_col='IC50_nM', corr_threshold=0.7):
    # надо было не склеивать сразу просто, но как вышло) 
    desc_only = df.drop(columns=[smiles_col, activity_col])

    # сразу отбираем только числовые признаки
    desc_only = desc_only.select_dtypes(include=[np.number])

    # убираем пропуски
    desc_only = desc_only.dropna(axis=1)

    # исключаем признаки с нулевой дисперсией
    selector = VarianceThreshold(threshold=0.0)
    desc_filtered = pd.DataFrame(
        selector.fit_transform(desc_only),
        columns=desc_only.columns[selector.get_support()]
    )


    # отбрасываем признаки с высокой корреляцией
    corr_matrix = desc_filtered.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [col for col in upper.columns if any(upper[col] > corr_threshold)]
    desc_filtered = desc_filtered.drop(columns=to_drop)

    # склеиваем обратно
    cleaned_df = pd.concat([df[[smiles_col, activity_col]].reset_index(drop=True),
                            desc_filtered.reset_index(drop=True)], axis=1)

    return cleaned_df

In [ ]:
# df_rdkit.shape

In [14]:
df_cleaned_rdkit = cleaning_desc(df_rdkit)
df_cleaned_rdkit.shape

(5098, 108)

In [15]:
df_cleaned_rdkit.head()

,canonical_smiles,IC50_nM,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,FpDensityMorgan1,AvgIpc,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,Cc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccccc1,60.00,11.568006,0.338694,-3.165650,0.736620,11.545455,311.406,0.818182,2.890001,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Cc1c(C=O)cc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,3230.00,13.231678,0.225436,-3.281084,0.667384,11.480000,357.406,1.000000,2.914829,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Cc1c(COc2cccc(Cl)c2)cc(-c2ccc(S(C)(=O)=O)cc2)n...,80.00,13.531957,0.252198,-3.293829,0.336163,11.500000,469.965,0.968750,3.085274,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Fc1ccc(-c2[nH]c(-c3ccc(F)cc3)c3c2C2CCC3CC2)cc1,0.12,13.353649,0.218100,-0.218100,0.558954,21.360000,335.397,0.600000,3.067789,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CCc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,100000.00,13.224282,0.274991,-3.216745,0.710290,11.625000,343.423,0.916667,2.986461,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
df_cleaned_rdkit.to_csv('cleaned_rdkit_data.csv', index=False)

In [17]:
df_mordred.shape

(5098, 1615)

In [18]:
df_cleaned_mordred = cleaning_desc(df_mordred)
df_cleaned_mordred.shape

#как-то сильно много удалилось. надеюсь, это норма)

(5098, 171)

In [19]:
df_cleaned_mordred.head()

,canonical_smiles,IC50_nM,nAcid,nBase,nAromAtom,nAtom,nSpiro,nBridgehead,nB,nN,...,n10FARing,n12FARing,JGI2,JGI3,JGI4,JGI6,JGI7,JGI8,JGI9,JGI10
0,Cc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccccc1,60.00,0.0,0.0,17.0,39.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.088889,0.053309,0.037292,0.016100,0.017601,0.021764,0.009167,0.008862
1,Cc1c(C=O)cc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,3230.00,0.0,0.0,17.0,41.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.094444,0.058943,0.048143,0.017395,0.019221,0.024418,0.009474,0.006762
2,Cc1c(COc2cccc(Cl)c2)cc(-c2ccc(S(C)(=O)=O)cc2)n...,80.00,0.0,0.0,23.0,53.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.082789,0.052288,0.046206,0.015421,0.016532,0.015410,0.006829,0.007751
3,Fc1ccc(-c2[nH]c(-c3ccc(F)cc3)c3c2C2CCC3CC2)cc1,0.12,0.0,0.0,17.0,44.0,0.0,2.0,0.0,1.0,...,0.0,0.0,0.072351,0.036499,0.037407,0.013961,0.015453,0.013958,0.013750,0.006612
4,CCc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,100000.00,0.0,0.0,17.0,42.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.093567,0.049342,0.041929,0.019151,0.019792,0.020901,0.012000,0.006762


In [20]:
df_cleaned_mordred.to_csv('cleaned_mordred_data.csv', index=False)

# 4. Вычислим фингерпринты

## morgan fp

In [21]:
def compute_morgan_fp(df, smiles_column='canonical_smiles', radius=2, n_bits=1024):
    # smiles to mol
    mols = [Chem.MolFromSmiles(smi) for smi in df[smiles_column]]

    # создаем генератор
    gen = rdFingerprintGenerator.GetMorganGenerator(radius=radius, fpSize=n_bits)

    # вычисляем фингерпринты
    fps = [gen.GetFingerprint(mol) for mol in mols]

    # в векторы
    fps_array = np.array([list(fp) for fp in fps])

    # в дфрейм
    fps_df = pd.DataFrame(fps_array, columns=[f'FP_bit_{i}' for i in range(n_bits)])

    # склеиваем
    df_fps = pd.concat([df.reset_index(drop=True), fps_df.reset_index(drop=True)], axis=1)

    return df_fps

In [22]:
df_cleaned_rdkit_morgan = compute_morgan_fp(df_cleaned_rdkit)
df_cleaned_rdkit_morgan.head()

,canonical_smiles,IC50_nM,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,FpDensityMorgan1,AvgIpc,...,FP_bit_1014,FP_bit_1015,FP_bit_1016,FP_bit_1017,FP_bit_1018,FP_bit_1019,FP_bit_1020,FP_bit_1021,FP_bit_1022,FP_bit_1023
0,Cc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccccc1,60.00,11.568006,0.338694,-3.165650,0.736620,11.545455,311.406,0.818182,2.890001,...,0,0,0,0,0,0,0,0,0,0
1,Cc1c(C=O)cc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,3230.00,13.231678,0.225436,-3.281084,0.667384,11.480000,357.406,1.000000,2.914829,...,0,0,0,0,0,0,0,0,0,0
2,Cc1c(COc2cccc(Cl)c2)cc(-c2ccc(S(C)(=O)=O)cc2)n...,80.00,13.531957,0.252198,-3.293829,0.336163,11.500000,469.965,0.968750,3.085274,...,0,0,0,0,0,0,0,0,0,0
3,Fc1ccc(-c2[nH]c(-c3ccc(F)cc3)c3c2C2CCC3CC2)cc1,0.12,13.353649,0.218100,-0.218100,0.558954,21.360000,335.397,0.600000,3.067789,...,0,0,0,0,0,1,0,0,0,0
4,CCc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,100000.00,13.224282,0.274991,-3.216745,0.710290,11.625000,343.423,0.916667,2.986461,...,0,0,0,0,0,0,0,0,0,0


In [23]:
df_cleaned_mordred_morgan = compute_morgan_fp(df_cleaned_mordred)
df_cleaned_mordred_morgan.head()

,canonical_smiles,IC50_nM,nAcid,nBase,nAromAtom,nAtom,nSpiro,nBridgehead,nB,nN,...,FP_bit_1014,FP_bit_1015,FP_bit_1016,FP_bit_1017,FP_bit_1018,FP_bit_1019,FP_bit_1020,FP_bit_1021,FP_bit_1022,FP_bit_1023
0,Cc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccccc1,60.00,0.0,0.0,17.0,39.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,Cc1c(C=O)cc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,3230.00,0.0,0.0,17.0,41.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,Cc1c(COc2cccc(Cl)c2)cc(-c2ccc(S(C)(=O)=O)cc2)n...,80.00,0.0,0.0,23.0,53.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,Fc1ccc(-c2[nH]c(-c3ccc(F)cc3)c3c2C2CCC3CC2)cc1,0.12,0.0,0.0,17.0,44.0,0.0,2.0,0.0,1.0,...,0,0,0,0,0,1,0,0,0,0
4,CCc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,100000.00,0.0,0.0,17.0,42.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
df_cleaned_rdkit_morgan.to_csv('df_cleaned_rdkit_morgan.csv', index=False)
df_cleaned_mordred_morgan.to_csv('df_cleaned_mordred_morgan.csv', index=False)

## maccs

In [25]:
def compute_maccs_fp(df, smiles_column='canonical_smiles'):
    # smiles в мол
    mols = [Chem.MolFromSmiles(smi) for smi in df[smiles_column]]

    # вычисляем maccs fp
    fps = [rdMolDescriptors.GetMACCSKeysFingerprint(mol) for mol in mols]

    # преобразуем в массив 0/1, игноря первый бит 
    fps_array = [[int(bit) for bit in fp.ToBitString()[1:]] for fp in fps]

    # в дфрейм
    fps_df = pd.DataFrame(fps_array, columns=[f'FP_MACCS_{i}' for i in range(1, 167)])

    # склеиваем
    df_fps = pd.concat([df.reset_index(drop=True), fps_df.reset_index(drop=True)], axis=1)

    return df_fps

In [26]:

df_cleaned_rdkit_maccs = compute_maccs_fp(df_cleaned_rdkit)
df_cleaned_rdkit_maccs.head()

,canonical_smiles,IC50_nM,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,FpDensityMorgan1,AvgIpc,...,FP_MACCS_157,FP_MACCS_158,FP_MACCS_159,FP_MACCS_160,FP_MACCS_161,FP_MACCS_162,FP_MACCS_163,FP_MACCS_164,FP_MACCS_165,FP_MACCS_166
0,Cc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccccc1,60.00,11.568006,0.338694,-3.165650,0.736620,11.545455,311.406,0.818182,2.890001,...,0,1,1,1,1,1,1,1,1,0
1,Cc1c(C=O)cc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,3230.00,13.231678,0.225436,-3.281084,0.667384,11.480000,357.406,1.000000,2.914829,...,0,1,1,1,1,1,1,1,1,0
2,Cc1c(COc2cccc(Cl)c2)cc(-c2ccc(S(C)(=O)=O)cc2)n...,80.00,13.531957,0.252198,-3.293829,0.336163,11.500000,469.965,0.968750,3.085274,...,1,1,1,1,1,1,1,1,1,0
3,Fc1ccc(-c2[nH]c(-c3ccc(F)cc3)c3c2C2CCC3CC2)cc1,0.12,13.353649,0.218100,-0.218100,0.558954,21.360000,335.397,0.600000,3.067789,...,0,0,0,0,1,1,1,0,1,0
4,CCc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,100000.00,13.224282,0.274991,-3.216745,0.710290,11.625000,343.423,0.916667,2.986461,...,0,1,1,1,1,1,1,1,1,0


In [27]:
df_cleaned_mordred_maccs = compute_maccs_fp(df_cleaned_mordred)
df_cleaned_mordred_maccs.head()

,canonical_smiles,IC50_nM,nAcid,nBase,nAromAtom,nAtom,nSpiro,nBridgehead,nB,nN,...,FP_MACCS_157,FP_MACCS_158,FP_MACCS_159,FP_MACCS_160,FP_MACCS_161,FP_MACCS_162,FP_MACCS_163,FP_MACCS_164,FP_MACCS_165,FP_MACCS_166
0,Cc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccccc1,60.00,0.0,0.0,17.0,39.0,0.0,0.0,0.0,1.0,...,0,1,1,1,1,1,1,1,1,0
1,Cc1c(C=O)cc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,3230.00,0.0,0.0,17.0,41.0,0.0,0.0,0.0,1.0,...,0,1,1,1,1,1,1,1,1,0
2,Cc1c(COc2cccc(Cl)c2)cc(-c2ccc(S(C)(=O)=O)cc2)n...,80.00,0.0,0.0,23.0,53.0,0.0,0.0,0.0,1.0,...,1,1,1,1,1,1,1,1,1,0
3,Fc1ccc(-c2[nH]c(-c3ccc(F)cc3)c3c2C2CCC3CC2)cc1,0.12,0.0,0.0,17.0,44.0,0.0,2.0,0.0,1.0,...,0,0,0,0,1,1,1,0,1,0
4,CCc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1,100000.00,0.0,0.0,17.0,42.0,0.0,0.0,0.0,1.0,...,0,1,1,1,1,1,1,1,1,0


In [28]:
df_cleaned_rdkit_maccs.to_csv('df_cleaned_rdkit_maccs.csv', index=False)
df_cleaned_mordred_maccs.to_csv('df_cleaned_mordred_maccs.csv', index=False)